right now, this compares 9 channel CNN vs. GLMs, in terms of raw performance (cc^2 or cc), on ALL neurons (so no subset stuff yet)

In [1]:
from collections import OrderedDict
from itertools import product
import h5py
import numpy as np
import pandas as pd
from tang_jcompneuro.model_fitting_postprocess import load_data_generic

In [2]:
def check_all(squared, score_col_name):
    models_to_examine = [
        ('cnn', 'b.9'),
        ('cnn', 'b.9', True),
        ('cnn', 'b.9', True, True),
        ('cnn', 'b.9_abs', True),
        ('cnn', 'b.9_avg_abs', True),
        ('cnn', 'mlp.4'),
#         ('cnn', 'mlp.4', True, True),
        ('cnn', 'mlp.20'),
#         ('cnn', 'mlp.20', True, True),
        ('cnn', 'mlp.40'),
#         ('cnn', 'mlp.40', True, True),
        ('cnn', 'mlp.60'),
        ('cnn', 'mlp.80'),
        ('cnn', 'mlp.100'),
        ('cnn', 'mlp.120'),
        ('cnn', 'mlp.145'),
        ('glm', 'linear_poisson'),
        ('glm', 'linear_softplus'),
        ('glm', 'fpower_poisson'),
#         ('glm', 'fpower_softplus'),
        ('glm', 'gqm.2_poisson'),
        ('glm', 'gqm.4_poisson'),
        ('glm', 'gqm.8_poisson'),
        ('glm', 'linear_gaussian'),
        ('glm', 'fpower_gaussian'),
        ('glm', 'gqm.2_gaussian'),
        ('glm', 'gqm.4_gaussian'),
        ('glm', 'gqm.8_gaussian'),
    ] # TODO should make this a ordered dict.
      # so that it will be easier to collect best val model for each model class.
    
    return load_data_generic(models_to_examine, load_naive=True, metric='raw', squared=squared,
                            score_col_name=score_col_name)

In [3]:
df_all_cc = check_all(squared=False, score_col_name='cc').xs(100, level='percentage').sort_index()

MkA_Shape all
MkA_Shape OT
MkE2_Shape all
MkE2_Shape OT


In [4]:
df_all_cc2 = check_all(squared=True, score_col_name='cc2').xs(100, level='percentage').sort_index()

MkA_Shape all
MkA_Shape OT
MkE2_Shape all
MkE2_Shape OT


In [5]:
# this is the way to change a multiindex's names.
# df_all.index.rename(['haha','haha2','haha3'], inplace=True)

In [6]:
# a = pd.concat([df_all_cc, df_all_cc2], axis=1, keys=[('1', 5), ('2', 3)], names=['df', '2', 'metric',])

In [7]:
a = pd.concat([df_all_cc, df_all_cc2], axis=1).sort_index()
a.columns.name = 'metric'
a.unstack('subset')

metric                                            cc                 cc2  \
subset                                            OT       all        OT   
dataset    model                                                           
MkA_Shape  cnn_b.9$False/False              0.338384  0.432367  0.160851   
           cnn_b.9$True/False               0.342747  0.438988  0.165052   
           cnn_b.9$True/True                0.363246  0.446413  0.176465   
           cnn_b.9_abs$True/False           0.331196  0.411028  0.154097   
           cnn_b.9_avg_abs$True/False       0.338470  0.411540  0.159901   
           cnn_mlp.100$False/False          0.260790  0.362913  0.108365   
           cnn_mlp.120$False/False          0.246871  0.361835  0.101883   
           cnn_mlp.145$False/False          0.244757  0.358066  0.099708   
           cnn_mlp.20$False/False           0.294273  0.378872  0.128778   
           cnn_mlp.4$False/False            0.223107  0.263226  0.079408   
           cnn_mlp.40$False/False           0.280815  0.379969  0.121556   
           cnn_mlp.60$False/False           0.270095  0.375731  0.114885   
           cnn_mlp.80$False/False           0.261192  0.368922  0.109129   
           glm_fpower_gaussian$False/False  0.260039  0.289380  0.101413   
           glm_fpower_poisson$False/False   0.261663  0.292536  0.103336   
           glm_gqm.2_gaussian$False/False   0.221168  0.322815  0.078149   
           glm_gqm.2_poisson$False/False    0.231374  0.338436  0.086884   
           glm_gqm.4_gaussian$False/False   0.235860  0.337893  0.088910   
           glm_gqm.4_poisson$False/False    0.244452  0.352186  0.096849   
           glm_gqm.8_gaussian$False/False   0.234528  0.333564  0.089052   
           glm_gqm.8_poisson$False/False    0.237862  0.346025  0.093232   
           glm_linear_gaussian$False/False  0.193430  0.225797  0.059773   
           glm_linear_poisson$False/False   0.201861  0.236187  0.066026   
           glm_linear_softplus$False/False  0.206024  0.238396  0.069433   
MkE2_Shape cnn_b.9$False/False              0.424903  0.533877  0.245155   
           cnn_b.9$True/False               0.428809  0.540964  0.250317   
           cnn_b.9$True/True                0.458771  0.551606  0.270537   
           cnn_b.9_abs$True/False           0.421803  0.515839  0.241657   
           cnn_b.9_avg_abs$True/False       0.428090  0.521616  0.247358   
           cnn_mlp.100$False/False          0.330182  0.459728  0.168910   
           cnn_mlp.120$False/False          0.319155  0.458433  0.162081   
           cnn_mlp.145$False/False          0.306130  0.452959  0.152036   
           cnn_mlp.20$False/False           0.377826  0.473496  0.203220   
           cnn_mlp.4$False/False            0.275746  0.317136  0.116267   
           cnn_mlp.40$False/False           0.366413  0.477765  0.198759   
           cnn_mlp.60$False/False           0.347722  0.472837  0.184963   
           cnn_mlp.80$False/False           0.334476  0.465872  0.175011   
           glm_fpower_gaussian$False/False  0.312055  0.348596  0.143512   
           glm_fpower_poisson$False/False   0.309706  0.351439  0.141858   
           glm_gqm.2_gaussian$False/False   0.295394  0.424180  0.135937   
           glm_gqm.2_poisson$False/False    0.307597  0.445536  0.146406   
           glm_gqm.4_gaussian$False/False   0.302531  0.437969  0.141875   
           glm_gqm.4_poisson$False/False    0.309054  0.457312  0.146710   
           glm_gqm.8_gaussian$False/False   0.292490  0.429254  0.133827   
           glm_gqm.8_poisson$False/False    0.295629  0.446494  0.136623   
           glm_linear_gaussian$False/False  0.248554  0.300218  0.099965   
           glm_linear_poisson$False/False   0.262213  0.320696  0.112176   

metric                                                
subset                                           all  
dataset    model                                      
MkA_Shape  cnn_b.9$False/False    